In [58]:
import numpy as np
# constant size
size = 1000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.1

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [59]:
n_states = 2
n_obs = 2

emission = np.random.rand(size,n_states)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))


forward = np.random.rand(size,n_states)
scale_factors = np.zeros((size))
forward_hat = np.zeros((size,n_states))

backward = np.random.rand(size,n_states)
backward_hat = np.zeros((size,n_states))

init_prob = np.array([0.5,0.5])
init_obs = np.zeros(n_states)

p_old = -10000
tol = 0.0001
max_iter = 100

mu = np.random.rand(n_states)

In [60]:
for ite in range(max_iter):
    forward[0,:] = init_prob * emission[0,:]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    
    for t in range(size-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[t+1,:]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        forward[t+1,:] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:]
    for t in reversed(range(size-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[t+1,:],np.transpose(transition))
        backward_hat[t,:] = temp/scale_factors[t+1]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((size,n_states))
    b = np.zeros((size,n_states,n_states))
    for i in range(size):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(size-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[np.argwhere(data==i),j] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    # if p - p_old < tol:
    #     break
    p_old = p

p is:-712.2057768636425
p is:-366.9245717940353
p is:-366.92422681632075
p is:-366.9241997771165
p is:-366.92417223859445
p is:-366.92414418717465
p is:-366.92411560903184
p is:-366.9240864900904
p is:-366.9240568160203
p is:-366.92402657223295
p is:-366.9239957438768
p is:-366.92396431583416
p is:-366.9239322727155
p is:-366.9238995988567
p is:-366.9238662783155
p is:-366.9238322948663
p is:-366.9237976319968
p is:-366.92376227290526
p is:-366.923726200495
p is:-366.92368939737264
p is:-366.92365184584366
p is:-366.92361352790886
p is:-366.9235744252626
p is:-366.9235345192875
p is:-366.92349379105366
p is:-366.9234522213146
p is:-366.9234097905055
p is:-366.9233664787391
p is:-366.92332226580584
p is:-366.92327713117015
p is:-366.9232310539694
p is:-366.92318401301185
p is:-366.9231359867763
p is:-366.9230869534092
p is:-366.9230368907264
p is:-366.92298577621034
p is:-366.92293358701113
p is:-366.9228802999464
p is:-366.9228258915024
p is:-366.9227703378334
p is:-366.92271361476514


In [61]:
transition

array([[0.28424844, 0.71575156],
       [0.47405953, 0.52594047]])

In [57]:
np.argwhere(data==1).shape

(282, 1)